In [1]:
import platform
import os
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

In [2]:
import numpy as np
from glob import glob
import pandas as pd
from tqdm import tqdm

In [3]:
os.chdir('../binary_one_dimension/')
from interfacegan_utils import calculate_boundary
os.chdir(current_wd)

In [4]:
latent_input_base_path = f"{DATA_PATH}/Models/InterfaceGAN/Inputs/entangled_typicality/"
all_paths = glob(f"{latent_input_base_path}/*")
all_paths = [path for path in all_paths if not "summary_stats" in path]
all_paths = [path for path in all_paths if not "typicality_scores" in path]

In [5]:
for path in tqdm(all_paths):
    out_dir = path.replace('Inputs', 'Outputs')
    if os.path.exists(out_dir):
        continue

    else:
        accuracies = []
        os.makedirs(out_dir, exist_ok=True)
        latent_codes_path = f"{path}/latents_concatenated.npy"
        scores_path = f"{path}/target.npy"
        out_name = f'boundary_dim.npy'
        acc = calculate_boundary(out_dir, out_name, latent_codes_path, scores_path)
        accuracies.append(acc)

        # save result
        n = path.split('/')[-1]

        # Crate dataframe with columns 'attribute', 'value1, 'value2', 'accuracy1', 'accuracy2', ...
        result = pd.DataFrame(columns=['n'] + ['acc'])
        result.loc[0] = [n] + accuracies
        result.to_csv(f"{out_dir}/summary_stats.csv", index=False)

100%|██████████| 5/5 [00:00<00:00, 2829.78it/s]


In [6]:
summary_paths = glob(f"{latent_input_base_path.replace('Inputs', 'Outputs')}/*/summary_stats.csv")

training_results = pd.concat([pd.read_csv(path) for path in summary_paths])
training_results.sort_values('n')

,n,acc
0,1000,0.981667
0,2000,0.982500
0,3000,0.952222
0,4000,0.933750
0,5000,0.907667
